In [1]:
import openturns as ot
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from shapley import ShapleyIndices, ShapleyKrigingIndices, SobolKrigingIndices
from shapley.tests import Ishigami, AdditiveGaussian
from shapley.plots import set_style_paper, plot_sensitivity_results
from shapley.base import ProbabilisticModel 
from shapley.kriging import KrigingModel

%load_ext autoreload
%autoreload 2

%matplotlib inline

set_style_paper()

savefigs = True
ylim = [-0.1, 1.]

In [2]:
mu_betas = [247, 218, 148, 105]
sigma_betas = [6, 5, 4, 3]
mu_normal = [35, 18, 14, 11, 8, 1, -31]
sigma_normal = [5, 5, 5, 5, 5, 5, 5]
bounds_low = [235, 207, 141, 100]
bounds_up = [259, 229, 155, 110]

margins = []
for mu, sigma, a, b in zip(mu_betas, sigma_betas, bounds_low, bounds_up):
    dist = ot.Beta()
    dist.setParameter(ot.BetaMuSigma()([mu, sigma, a, b]))
    margins.append(dist)
    
for mu, sigma in zip(mu_normal, sigma_normal):
    margins.append(ot.Normal(mu, sigma))
    
mu = np.asarray(mu_betas + mu_normal)
sigma = np.asarray(sigma_betas + sigma_normal)
dim = len(mu)

In [3]:
correlation = ot.CorrelationMatrix(pd.read_csv('./data/correlation.csv', sep=',', header=None).values)
copula = ot.NormalCopula(correlation)
input_distribution = ot.ComposedDistribution(margins, copula)

In [4]:
data = pd.read_csv('./data/results_unif.txt', sep=' ', header=None, usecols=range(dim+1))
X = data.values[:, :dim]
y = data.values[:, 11]
X = (X - mu) / sigma
y = (y - y.mean()) / y.std()

In [7]:
library = 'sklearn'
basis = 'quadratic'
kernel = 'matern'

meta_model = KrigingModel(model=lambda x: None, input_distribution=input_distribution)
meta_model.input_sample = X
meta_model.output_sample = y
meta_model.build(library=library, basis_type=basis, kernel=kernel)
score_q2_loo = meta_model.compute_score_q2_loo()
print('Leave One Out Q2 score:', score_q2_loo)

In [9]:
sobol_kriging = SobolKrigingIndices(meta_model.input_distribution)


In [19]:
%%time
n_sample = 1000
n_realization = 300

# Evaluate the kriging model
sobol_kriging.build_uncorrelated_mc_sample(meta_model, n_sample, n_realization)

TypeError: InvalidArgumentException : The correlation matrix must be definite positive R=class=CorrelationMatrix dimension=11 implementation=class=MatrixImplementation name=Unnamed rows=11 columns=11 values=[1,0.949053,0,0,0,0,0.799513,0.641382,0.247935,-0.253111,0.957044,0.949053,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.738845,0.692445,0.310764,0.226803,0.196703,0.799513,0,0,0,0,0.738845,1,0.531112,0.593523,0.509915,0.458519,0.641382,0,0,0,0,0.692445,0.531112,1,0.599338,0.571526,0.543489,0.247935,0,0,0,0,0.310764,0.593523,0.599338,1,-0.346391,-0.332602,-0.253111,0,0,0,0,0.226803,0.509915,0.571526,-0.346391,1,0.840811,0.957044,0,0,0,0,0.196703,0.458519,0.543489,-0.332602,0.840811,1]

In [ ]:
%%time
shapley_kriging.build_mc_sample(meta_model, n_perms=n_perms, Nv=Nv, No=No, Ni=Ni, n_realization=n_realization)

In [27]:
1001000 % 2000

1000

In [ ]:
%%time
n_boot = 1000
shapley_kriging_results = shapley_kriging.compute_indices(n_boot=n_boot)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
plot_sensitivity_results(shapley_kriging_results, indice='all', ax=ax)
ax.set_ylim(ylim)
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 8), sharex=True, sharey=True)

ax = axes[0]
plot_sensitivity_results(shapley_kriging_results, indice='first', ax=ax)
ax.set_ylim(ylim)
ax.set_xlabel('')
ax.set_ylabel('First Sobol indices')
ax.legend(loc=2)

ax = axes[1]
plot_sensitivity_results(shapley_kriging_results, indice='total', ax=ax)
ax.set_ylim(ylim)
ax.legend(loc=2)
ax.set_xlabel('')
ax.set_ylabel('Total Sobol indices')

ax = axes[2]
plot_sensitivity_results(shapley_kriging_results, indice='shapley', ax=ax)
ax.set_ylim(ylim)
ax.legend(loc=2)
ax.set_ylabel('Shapley effects')

fig.tight_layout()